In [1]:
import numpy as np
import pandas as pd

def tasaCupon0(datos):
    """ Retorna una lista con las tasas cupón 0
    PARAMETROS
    ==========
    datos: una lista que contiene tuplas de: precio,cupón
            Ej: datos = [(97,0.5),(97,0.5),(98.7,0),(101.3,4),(100,4.5)]
    """
    r0 = []
    for per,dato in enumerate(datos):
        r = np.rate(per+1, 0, -dato[0]-sum([np.pv(r0[i], i+1, 0, dato[1]) for i in range(per)]), 1+dato[1])
        r0.append(r)
    return r0

def tasaFwd(Tasa0, all = True):
    fwd = []
    if all == True:
        for i in range(len(Tasa0)):
            r = ((1+Tasa0[i])**(i+1))/((1+Tasa0[i-1])**(i))-1
            fwd.append(float(format(r,'.4f')))
        return fwd

    elif all == False:
        i = int(input('Ingrese periodo inicial:'))
        f = int(input('Ingrese periodo final:'))
        r = (((1+Tasa0[f])**f)/((1+Tasa0[i])**i))**(f-i)**-1-1
        return format(r,'.2%')

def sensibilidad(vn, tmar, cupon, plazo, var_i):
    va = [-np.pv(tmar,i,0,cupon) for i in range(1,plazo+1)]
    va[-1] -= np.pv(tmar,plazo,0,1)
    x = va/sum(va)

    duracion = sum([(i+1)*j for i,j in enumerate(x)])
    convexidad = sum([((i+1)*(i+2)*j)/(1+tmar)**2 for i,j in enumerate(x)])
    dm = duracion/(1+tmar)
    eDuracion = var_i*-dm
    eConvexidad = 0.5*convexidad*var_i**2
    total = eDuracion+eConvexidad
    nuevoPrecio = np.pv(tmar,plazo,cupon*vn,vn)*(1+total)

    print(31*'=')
    print('{:20} {:10.2f}'.format('Duración', duracion))
    print('{:20} {:10.2f}'.format('Duración Modificada', dm))
    print('{:20} {:10.2f}'.format('Convexidad', convexidad))
    print('EFECTOS'.center(31,'-'))
    print('{:20} {:>10.2%}'.format('Duración', eDuracion))
    print('{:20} {:>10.2%}'.format('Convexidad', eConvexidad))
    print('{:20} {:>10.2%}'.format('Total', total))
    print(31*'-')
    print('{:20} {:>10.2f}'.format('Precio Nuevo', nuevoPrecio))
    print(31*'=')

def generarReporteBonos(Precios, Cupones):
    tupla = ((precio,cupon) for precio,cupon in zip(precios, cupones))
    Cupon0 = tasaCupon0(tupla)
    Forward = tasaFwd(Cupon0)

    print(35*'=')
    print('{:<5} {:10} {:10} {:10}'.format('t', 'TIR=CUPON', 'CUPON 0', 'FORWARD'))
    print(35*'=')
    for i in range(len(Cupones)):
        print('{:<5} {:^10.2%} {:^10.2%} {:^10.2%}'.format(i+1, Cupones[i], Cupon0[i], Forward[i]))
    print(35*'=')

def convertirTasa(Tasa):
    """Devuelve un cuadro de tasas equivalentes"""
    per = 'Anual', 'Semestral', 'Quimestral', 'Cuatrimestral', 'Trimestral', 'Bimestral', 'Mensual', 'Bimensual', 'Semanal', 'Diaria'
    con = 1,2,12/5,3,4,6,12,24,52,360
    print('Seleccione el período de la tasa:')
    for i,j in enumerate(per):
        print('{} {}'.format('('+str(i)+')',j),end=' ')

    opcion = int(input('\nElija una opción:'))
    n = con[opcion]

    print('''
    (0) Efectiva
    (1) Nominal''')
    opcion2 = int(input('Elija una opción:'))

    if opcion2 == 0:
        m = n
        s = 'Tasa: {:.2%} efectiva {}'.format(Tasa,per[opcion].lower())
    elif opcion2 == 1:
        print('\nSeleccione el período de capitalización:')
        for i,j in enumerate(per):
            print('{} {}'.format('('+str(i)+')',j),end=' ')
        opcion3 = int(input('\nElija una opción:'))
        m = con[opcion3]
        s = 'Tasa: {:.2%} {} comp. {}'.format(Tasa,per[opcion].lower(), per[opcion3].lower())

    print('\n'+s)
    print(113*'=')
    print(('{:15} '+11*'{:^8.5s} ').format('Períodos','Efect.',*(i for i in per)))
    print(113*'=')
    for i in range(len(per)):
        s = 11*'{:>8.3%} '
        tasasNom = (con[i]/j*((1+Tasa*n/m)**(m/con[i])-1) for j in con)
        tasasCont = (np.log1p(Tasa*n/m)*m/j for j in con)
        efe = (1+Tasa*n/m)**(m/con[i])-1
        print(('{:15}'+s).format(per[i], efe, *tasasNom))
    print(('{:15}'+s).format('Continua', 0, *tasasCont))
    print(113*'=')

def tasas(Tasa, PerTasa, PerCap):
    """Devuelve un cuadro de tasas equivalentes,
        a diferencia de la convertirTasa, no contiene menus asistidos
        ya que toda la información se la pone en los argumentos de la función
    """

    per = 'Anual', 'Semestral', 'Quimestral', 'Cuatrimestral', 'Trimestral', 'Bimestral', 'Mensual', 'Bimensual', 'Semanal', 'Diaria'
    con = 1,2,12/5,3,4,6,12,24,52,360

    n = PerTasa
    m = PerCap
    

    if n==m:
        s = 'Tasa: {:.2%} efectiva {}'.format(Tasa,per[con.index(n)].lower())
    else:
        s = 'Tasa: {:.2%} {} comp. {}'.format(Tasa,per[con.index(n)].lower(), per[con.index(m)].lower())

    print('\n'+s)

    mat = []
    for i in range(len(per)):
        tasasNom = [con[i]/j*((1+Tasa*n/m)**(m/con[i])-1) for j in con]
        tasasCont = [np.log1p(Tasa*n/m)*m/j for j in con]
        efe = (1+Tasa*n/m)**(m/con[i])-1
        fila = mat.append([efe] + tasasNom)
    mat = np.array(mat)
    pd.options.display.float_format = '{:,.2%}'.format
    df = pd.DataFrame(mat,per,['Efect']+[i[:5] for i in per])
    return df

def tasaEquivalente(tasa1, tasa2):
    """Convierte a una tasa equivalente tasa1=[tasa, perTasa, Efect=True, perCap], tasa2=[perTasa, Efect=True, perCap]"""
    per = 'Anual', 'Semestral', 'Quimestral', 'Cuatrimestral', 'Trimestral', 'Bimestral', 'Mensual', 'Bimensual', 'Semanal', 'Diaria'
    con = 1,2,12/5,3,4,6,12,24,52,360
    if tasa1[2]:
        n = m = tasa1[1]
        s1 = '{:.2%} efectiva {}'.format(tasa1[0],per[con.index(n)].lower())
    else:
        n = tasa1[1]
        m = tasa1[3]
        s1 = '{:.2%} {} comp. {}'.format(tasa1[0],per[con.index(n)].lower(), per[con.index(m)].lower())

    if tasa2[1]:
        n2 = tasa2[0]
        r = (1+tasa1[0]*n/m)**(m/n2)-1
        s2 = '{:.2%} efectiva {}'.format(r,per[con.index(n2)].lower())
    else:
        n2 = tasa2[0]
        m2 = tasa2[2]
        r = m2/n2*((1+tasa1[0]*n/m)**(m/m2)-1)
        s2 = '{:.2%} {} comp. {}'.format(r,per[con.index(n2)].lower(), per[con.index(m2)].lower())
    return '{} == {}'.format(s1,s2)

In [2]:
convertirTasa(0.01)

Seleccione el período de la tasa:
(0) Anual (1) Semestral (2) Quimestral (3) Cuatrimestral (4) Trimestral (5) Bimestral (6) Mensual (7) Bimensual (8) Semanal (9) Diaria 


Elija una opción: 6



    (0) Efectiva
    (1) Nominal


Elija una opción: 1



Seleccione el período de capitalización:
(0) Anual (1) Semestral (2) Quimestral (3) Cuatrimestral (4) Trimestral (5) Bimestral (6) Mensual (7) Bimensual (8) Semanal (9) Diaria 


Elija una opción: 3



Tasa: 1.00% mensual comp. cuatrimestral
Períodos         Efect    Anual    Semes    Quime    Cuatr    Trime    Bimes    Mensu    Bimen    Seman    Diari   
Anual           12.486%  12.486%   6.243%   5.203%   4.162%   3.122%   2.081%   1.041%   0.520%   0.240%   0.035% 
Semestral        6.060%  12.119%   6.060%   5.050%   4.040%   3.030%   2.020%   1.010%   0.505%   0.233%   0.034% 
Quimestral       5.025%  12.059%   6.030%   5.025%   4.020%   3.015%   2.010%   1.005%   0.502%   0.232%   0.033% 
Cuatrimestral    4.000%  12.000%   6.000%   5.000%   4.000%   3.000%   2.000%   1.000%   0.500%   0.231%   0.033% 
Trimestral       2.985%  11.941%   5.970%   4.975%   3.980%   2.985%   1.990%   0.995%   0.498%   0.230%   0.033% 
Bimestral        1.980%  11.882%   5.941%   4.951%   3.961%   2.971%   1.980%   0.990%   0.495%   0.229%   0.033% 
Mensual          0.985%  11.824%   5.912%   4.927%   3.941%   2.956%   1.971%   0.985%   0.493%   0.227%   0.033% 
Bimensual        0.491%  11.795%   5.8

In [5]:
tasas(0.01, 12,3)


Tasa: 1.00% mensual comp. cuatrimestral


,Efect,Anual,Semes,Quime,Cuatr,Trime,Bimes,Mensu,Bimen,Seman,Diari
Anual,12.49%,12.49%,6.24%,5.20%,4.16%,3.12%,2.08%,1.04%,0.52%,0.24%,0.03%
Semestral,6.06%,12.12%,6.06%,5.05%,4.04%,3.03%,2.02%,1.01%,0.50%,0.23%,0.03%
Quimestral,5.02%,12.06%,6.03%,5.02%,4.02%,3.01%,2.01%,1.00%,0.50%,0.23%,0.03%
Cuatrimestral,4.00%,12.00%,6.00%,5.00%,4.00%,3.00%,2.00%,1.00%,0.50%,0.23%,0.03%
Trimestral,2.99%,11.94%,5.97%,4.98%,3.98%,2.99%,1.99%,1.00%,0.50%,0.23%,0.03%
Bimestral,1.98%,11.88%,5.94%,4.95%,3.96%,2.97%,1.98%,0.99%,0.50%,0.23%,0.03%
Mensual,0.99%,11.82%,5.91%,4.93%,3.94%,2.96%,1.97%,0.99%,0.49%,0.23%,0.03%
Bimensual,0.49%,11.80%,5.90%,4.91%,3.93%,2.95%,1.97%,0.98%,0.49%,0.23%,0.03%
Semanal,0.23%,11.78%,5.89%,4.91%,3.93%,2.94%,1.96%,0.98%,0.49%,0.23%,0.03%
Diaria,0.03%,11.77%,5.88%,4.90%,3.92%,2.94%,1.96%,0.98%,0.49%,0.23%,0.03%


In [4]:
tasaEquivalente([0.01,52,52],[6,6])

'1.00% efectiva semanal == 9.01% efectiva bimestral'